# Классификатор

## Подготовка данных 

In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torchmetrics

from torch import nn
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter

from model import Autoencoder, Classifier
from utils import plot_reconstructed, grid_plot, vis_confusion




class Encode:
    def __init__(self):
        self.autoencoder_weights = 'outputs/autoencoder_model.pth'
        self._autoencoder = Autoencoder()
        self._autoencoder.load_state_dict(torch.load(self.autoencoder_weights))

        self.encoder = self._autoencoder.encoder
        
    def __call__(self, image: torch.Tensor) -> torch.Tensor:
        return self.encoder(image)


transform = transforms.Compose([
    transforms.ToTensor(),
    Encode(),
    transforms.Lambda(torch.flatten)
])


data = CIFAR10('data/', train=False, download=True, transform=transform)
# data = CIFAR10('data/', train=False, download=True)

data_loader = torch.utils.data.DataLoader(dataset=data, batch_size=16, shuffle=True)

Files already downloaded and verified


In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

classes = {label: i for i, label in enumerate(classes)}

In [ ]:
validate

random_split

In [2]:
data[0][0].shape

torch.Size([256])

## Обучение классификатора

In [3]:
net = Classifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)


num_epochs = 1
tb = SummaryWriter()

for epoch in range(num_epochs):
    train_loss = 0.0
    for batch, labels in data_loader:
        outputs = net(batch)
        
        loss = criterion(outputs, labels)
        tb.add_scalar("Classifier: Loss/train", loss, epoch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Validate model every N epoch
    if epoch%10 == 0:
        

    # # Log results with a last batch from the loop
    # grid = grid_plot(batch, reconstructed)
    # tb.add_image('Original vs Reconstructed', grid, epoch)


In [ ]:
import numpy as np
epochs = 5
min_valid_loss = np.inf

for e in range(epochs):
    train_loss = 0.0
    model.train()     # Optional when not using Model Specific layer
    for data, labels in trainloader:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        
        optimizer.zero_grad()
        target = model(data)
        loss = criterion(target,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    valid_loss = 0.0
    model.eval()     # Optional when not using Model Specific layer
    for data, labels in validloader:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        
        target = model(data)
        loss = criterion(target,labels)
        valid_loss = loss.item() * data.size(0)

    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(trainloader)} \t\t Validation Loss: {valid_loss / len(validloader)}')
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(model.state_dict(), 'saved_model.pth')

In [4]:
outputs = net(batch)
_, y_pred = torch.max(outputs, 1)


print(labels)
print(y_pred)

tensor([8, 3, 6, 2, 1, 2, 0, 8, 0, 9, 9, 7, 9, 7, 0, 3])
tensor([9, 6, 4, 6, 8, 6, 2, 4, 5, 9, 8, 9, 9, 8, 0, 4])


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = net.to(device)


num_epochs = 10

for epoch in range(num_epochs):
    logs = {}
    for phase in ['train', 'validation']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            if phase == 'train':
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.detach() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_corrects.float() / len(dataloaders[phase].dataset)
        
        prefix = ''
        if phase == 'validation':
            prefix = 'val_'

        logs[prefix + 'log loss'] = epoch_loss.item()
        logs[prefix + 'accuracy'] = epoch_acc.item()

In [13]:
torch.save(net.state_dict(), 'outputs/clf_model.pth')

## Оценка качества

In [5]:
net = Classifier()
net.load_state_dict(torch.load('outputs/clf_model.pth'))

<All keys matched successfully>

In [5]:
outputs = net(batch)
_, y_pred = torch.max(outputs, 1)


print(labels)
print(y_pred)

tensor([8, 3, 6, 2, 1, 2, 0, 8, 0, 9, 9, 7, 9, 7, 0, 3])
tensor([9, 6, 4, 6, 8, 6, 2, 4, 5, 9, 8, 9, 9, 8, 0, 4])


In [6]:


matrix = torchmetrics.ConfusionMatrix('multiclass', num_classes=10)

matrix = matrix(y_pred, labels).numpy()


vis_confusion(tb, 1, matrix, classes)



In [ ]:
tb = SummaryWriter()



